In [5]:
import pandas as pd
df = pd.read_csv("data.csv")
df.head(5)

,Unnamed: 0,ProductId,ProductName,Branch,level_1,level_2,level_3,PricePerItem,PriceMode,HasMultiBuyDeal,MultiBuyDeal,MultiBuyBasePrice,MultiBuyPrice,MultiBuyQuantity,PromoBadgeImageLabel
0,0,5000961-EA-000NW,Johnson's Baby Lotion,New World Mt Roskill,"Baby, Toddler & Kids",Baby Care,Baby Bathing & Skin Care,4.89,ea,False,NaN,4.89,NaN,NaN,NaN
1,1,5000962-EA-000NW,Johnson's Baby Oil,New World Mt Roskill,"Baby, Toddler & Kids",Baby Care,Baby Bathing & Skin Care,4.89,ea,False,NaN,4.89,NaN,NaN,NaN
2,2,5000963-EA-000NW,SMA Balanced Nutrition From Birth Infant Formula,New World Mt Roskill,"Baby, Toddler & Kids",Baby Care,Baby Formula & Toddler Food,17.99,ea,False,NaN,17.99,17.99,1.0,Saver
3,3,5000965-EA-000NW,Johnson's Baby Powder,New World Mt Roskill,"Baby, Toddler & Kids",Baby Care,Nappies & Changing,2.99,ea,False,NaN,2.99,2.99,1.0,Saver
4,4,5000966-EA-000NW,Johnson's Baby Powder,New World Mt Roskill,"Baby, Toddler & Kids",Baby Care,Nappies & Changing,4.89,ea,False,NaN,4.89,NaN,NaN,NaN


In [6]:
columns = ["ProductId", "ProductName", "Branch", "Level1", "Level2", "Level3", "PricePerItem", "PriceMode", "HasMultiBuyDeal", "MultiBuyDeal", "MultiBuyBasePrice", "MultiBuyPrice", "MultiBuyQuantity", "PromoBadgeImageLabel"]
df = pd.read_csv("data.csv", index_col=0)
df.head(5)


,ProductId,ProductName,Branch,level_1,level_2,level_3,PricePerItem,PriceMode,HasMultiBuyDeal,MultiBuyDeal,MultiBuyBasePrice,MultiBuyPrice,MultiBuyQuantity,PromoBadgeImageLabel
0,5000961-EA-000NW,Johnson's Baby Lotion,New World Mt Roskill,"Baby, Toddler & Kids",Baby Care,Baby Bathing & Skin Care,4.89,ea,False,NaN,4.89,NaN,NaN,NaN
1,5000962-EA-000NW,Johnson's Baby Oil,New World Mt Roskill,"Baby, Toddler & Kids",Baby Care,Baby Bathing & Skin Care,4.89,ea,False,NaN,4.89,NaN,NaN,NaN
2,5000963-EA-000NW,SMA Balanced Nutrition From Birth Infant Formula,New World Mt Roskill,"Baby, Toddler & Kids",Baby Care,Baby Formula & Toddler Food,17.99,ea,False,NaN,17.99,17.99,1.0,Saver
3,5000965-EA-000NW,Johnson's Baby Powder,New World Mt Roskill,"Baby, Toddler & Kids",Baby Care,Nappies & Changing,2.99,ea,False,NaN,2.99,2.99,1.0,Saver
4,5000966-EA-000NW,Johnson's Baby Powder,New World Mt Roskill,"Baby, Toddler & Kids",Baby Care,Nappies & Changing,4.89,ea,False,NaN,4.89,NaN,NaN,NaN
